# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug
# !pip install torchvision

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)


In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import os
from dataclasses import dataclass
from IPython.display import HTML

import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.debugger import FrameworkProfile
from sagemaker.debugger import ProfilerConfig
from sagemaker.debugger import ProfilerRule
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import CategoricalParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import HyperparameterTuner
from sagemaker.tuner import IntegerParameter
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

[2021-11-26 23:25:06.958 datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395:18 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


In [4]:
sess = sagemaker.Session()

## `Configuration`

In [5]:
@dataclass
class Config:
    BUCKET = sess.default_bucket()
    REGION = sess.boto_region_name
    EXECUTION_ROLE = sagemaker.get_execution_role()
    SEED = 23
    INSTANCE_COUNT = 1
    INSTANCE_TYPE = "ml.p2.xlarge"
    MAX_JOBS = 4
    MAX_PARALLEL_JOBS = 1
    MONITORING_INTERVAL_MILLIS = 100
    PROFILE_EVERY_X_STEPS = 10

In [6]:
config = Config()

In [7]:
os.environ["DEFAULT_S3_BUCKET"] = f"s3://{config.BUCKET}/"

## `Dataset`
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [8]:
# Download and unzip dataset
!mkdir data
!wget https://github.com/DataScienceNigeria/Research-Papers-by-Data-Science-Nigeria/raw/master/AFRIFASHION1600:%20A%20Contemporary%20African%20Fashion%20Dataset%20for%20Computer%20Vision/data/AFRIFASHION1600.zip -P data
!unzip data/AFRIFASHION1600.zip -d data && rm data/AFRIFASHION1600.zip

--2021-11-26 00:47:29--  https://github.com/DataScienceNigeria/Research-Papers-by-Data-Science-Nigeria/raw/master/AFRIFASHION1600:%20A%20Contemporary%20African%20Fashion%20Dataset%20for%20Computer%20Vision/data/AFRIFASHION1600.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataScienceNigeria/Research-Papers-by-Data-Science-Nigeria/master/AFRIFASHION1600%3A%20A%20Contemporary%20African%20Fashion%20Dataset%20for%20Computer%20Vision/data/AFRIFASHION1600.zip [following]
--2021-11-26 00:47:29--  https://raw.githubusercontent.com/DataScienceNigeria/Research-Papers-by-Data-Science-Nigeria/master/AFRIFASHION1600%3A%20A%20Contemporary%20African%20Fashion%20Dataset%20for%20Computer%20Vision/data/AFRIFASHION1600.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.1

In [9]:
# Split dataset into train and validation sets
!python oge/dataset.py --dataset_path data/AFRIFASHION1600 --train_size 0.7 #--seed {c.SEED}

In [10]:
# Upload dataset to S3
!mkdir -p logs
!aws s3 sync data/AFRIFASHION1600 ${DEFAULT_S3_BUCKET}afrifashion1600/ > logs/data_upload.log 2>&1

In [11]:
# Confirm dataswet upload to S3
!aws s3 ls ${DEFAULT_S3_BUCKET}afrifashion1600/

                           PRE train/
                           PRE val/


## `Hyperparameter Tuning`
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [12]:
# TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "train_batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
    "lr": ContinuousParameter(0.001, 0.1),
    "n_epochs": CategoricalParameter([3, 5, 7]),
    "normalize": CategoricalParameter([True, False])
}

In [13]:
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{
    "Name": "average test loss",
    "Regex": "Test set: Average loss: ([0-9\\.]+)"
}]

In [20]:
estimator = PyTorch(
    source_dir = "oge",
    entry_point="train_model.py",
    base_job_name="oge-afrifashion-resnet50-pytorch",
    role=config.EXECUTION_ROLE,
    instance_count=config.INSTANCE_COUNT,
    instance_type=config.INSTANCE_TYPE,   # TODO
    framework_version="1.8",
    py_version="py36"
)

In [21]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=config.MAX_JOBS,
    max_parallel_jobs=config.MAX_PARALLEL_JOBS,   
    objective_type=objective_type
)

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit(
    {
        "train": "s3://sagemaker-us-east-1-661396466295/afrifashion1600/train",
        "test": "s3://sagemaker-us-east-1-661396466295/afrifashion1600/val"
    }
)

............................................................................................................................................................................................................................................................................................................................................................................

In [24]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator() #TODO

#Get the hyperparameters of the best trained model
best_hyperparameters = best_estimator.hyperparameters()
print(best_hyperparameters)


2021-11-27 00:49:38 Starting - Preparing the instances for training
2021-11-27 00:49:38 Downloading - Downloading input data
2021-11-27 00:49:38 Training - Training image download completed. Training in progress.
2021-11-27 00:49:38 Uploading - Uploading generated training model
2021-11-27 00:49:38 Completed - Training job completed
{'_tuning_objective_metric': '"average test loss"', 'lr': '0.0015630256742041576', 'n_epochs': '"5"', 'normalize': '"True"', 'sagemaker_container_log_level': '20', 'sagemaker_estimator_class_name': '"PyTorch"', 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"', 'sagemaker_job_name': '"oge-afrifashion-resnet50-pytorch-2021-11-27-00-14-36-639"', 'sagemaker_program': '"train_model.py"', 'sagemaker_region': '"us-east-1"', 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-661396466295/oge-afrifashion-resnet50-pytorch-2021-11-27-00-14-36-639/source/sourcedir.tar.gz"', 'train_batch_size': '"32"'}


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
MONITORING_INTERVAL_MILLIS = 100
PROFILE_EVERY_X_STEPS = 10

In [ ]:
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=config.MONITORING_INTERVAL_MILLIS,
    framework_profile_params=FrameworkProfile(num_steps=config.PROFILE_EVERY_X_STEPS)
)

In [ ]:
hyperparameters = {
    "train_batch_size": 64,
    "test_batch_size": 64,
    "n_epochs": 2,
    
}

In [ ]:
# TODO: Create and fit an estimator
estimator = PyTorch(
    source_dir="oge",
    entry_point="train_model.py",
    base_job_name="oge-afrifashion-resnet50-pytorch",
    role=get_execution_role(),
    instance_count=config.INSTANCE_COUNT,
    instance_type=config.INSTANCE_TYPE,   # TODO
    framework_version="1.8",
    py_version="py36",
    profiler_config=profiler_config,
    rules=rules
)

In [ ]:
estimator.fit(wait=True)

In [ ]:
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {config.REGION}")

In [ ]:
training_job = TrainingJob(training_job_name, config.REGION)
training_job.wait_for_sys_profiling_data_to_be_available()

In [ ]:
# TODO: Plot a debugging output.
system_metrics_reader = training_job.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["GPU", "CPU"],
    select_event=["total"]
)

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(rule_output_path)

In [ ]:
!aws s3 ls {rule_output_path} --recursive

In [ ]:
!aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
profile_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [ ]:
HTML(filename=profile_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = 0# TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()